<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E4%B8%AD%E6%96%87%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96%E6%B1%87%E6%80%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/CardBuild/dialogue_extract/0120res*

/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_0_to_10.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_10146_to_10870.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_10870_to_11594.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_10_to_734.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_11594_to_12318.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_1206_to_1809.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_12318_to_13042.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_12663_to_13266.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_13042_to_13766.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_13766_to_14490.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_1458_to_2182.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_15214_to_15938.txt
/content/drive/MyDrive/CardBuild/dialogue_extract/0120res_15938_to_16662.txt
/content/drive/

In [ ]:
import os
import json

# 指定文件夹路径
save_path = "/content/drive/MyDrive/CardBuild/dialogue_extract"

all_datas = []

count = 0
parsing_error_count = 0

# 遍历指定文件夹中的所有txt文件
for filename in os.listdir(save_path):
    if filename.endswith(".txt") and filename.startswith("0120res"):
        # 构建完整的文件路径
        file_path = os.path.join(save_path, filename)

        error_count = 0

        # 打开并读取文件
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if error_count >3:
                    break
                try:
                    # 将每行解析为JSON
                    data = json.loads(line)

                    if "input" in data and "response" in data and "last_response" in data and "last_input" in data and \
                        "id" in data and "source" in data and "last_id" in data:
                        all_datas.append({
                            "source": data["source"],
                            "id": data["last_id"],
                            "input": data["last_input"],
                            "response": data["last_response"]
                        })

                        response_str = data['response']
                        try:
                            response = json.loads(response_str)
                            len_prefix = len("input paragraph:\n")
                            real_input = data["input"][len_prefix:]
                            all_datas.append({
                                "source": data["source"],
                                "id": data["id"],
                                "input": real_input,
                                "response": response
                            })

                        except json.JSONDecodeError:
                            parsing_error_count += 1
                            continue

                        pass
                        count += 1
                    else:
                        error_count += 1
                        continue

                    continue

                    response = json.loads(response_str)
                    last_summary = response['summary'].strip(' -')
                    if "input" in data:
                        last_input = data['input']
                    else:
                        last_input = long_text_to_input(data["text"])

                    last_input = last_input[len_prefix:]
                    rest_paragraph = data['text'][len(last_input):].strip()
                    next_input = long_text_to_input(last_summary + "\n" + rest_paragraph)

                    all_datas.append({
                        "source": data["source"],
                        "last_id": data["id"],
                        "id": data["id"] + "_2nd_seg",
                        "input": last_summary,
                        "last_input": last_input,
                        "last_response": response,
                        "next_input": next_input
                    })
                    # break
                except json.JSONDecodeError:
                    # 如果解析JSON失败，跳过这一行
                    error_count += 1
                    continue


        # break

# 输出结果



In [ ]:
task_prompt = """给定input paragraph，抽取其中的对话，并输出为json格式

Let's think it step by step
1. summarize input paragraph into bullet format，存储在summary字段
2. 抽取每一句对话的内容 dialogue，判断每一句话的说话人 said by, 存储在conversations中"""


In [ ]:
print(len(all_datas))

67974


In [ ]:
print(all_datas[0]['response'])
print(all_datas[1]['response'])
print(all_datas[0].keys())

{'summary': '- 夏子明要求单翔重复之前说过的话，然后要求黎明给他做饭，最终吃了两份饭菜，引发了关于打打杀杀生活选择的对话。夏子明最终以吃了两碗饭为胜利理由，要求黎明认输。', 'conversations': [{'dialogue': '你把刚才的话再重复一遍', 'said_by': '夏子明'}, {'dialogue': '就是上一句话', 'said_by': '夏子明'}, {'dialogue': '再上一句话', 'said_by': '夏子明'}, {'dialogue': '我想要黎哥哥给我做饭吃，我饿了', 'said_by': '单翔'}, {'dialogue': '我真不明白，就凭你做饭的这份厨艺，到哪儿都会混得很好，为什么你要选择这样打打杀杀的', 'said_by': '夏子明'}, {'dialogue': '曾经也有人这样说过', 'said_by': '黎明'}, {'dialogue': '那你现在还是过着这种打打杀杀的生活', 'said_by': '夏子明'}, {'dialogue': '因为，在这个世界，像你们这样爱吃霸王餐的人，实在是太多了', 'said_by': '黎明'}, {'dialogue': '再来一碗', 'said_by': '夏子明'}, {'dialogue': '我也还要一碗', 'said_by': '单翔'}, {'dialogue': '剩下的全都在他碗里去了', 'said_by': '黎明'}, {'dialogue': '你能把你的，分给我一半吗', 'said_by': '单翔'}, {'dialogue': '不行', 'said_by': '夏子明'}, {'dialogue': '别忘了，今后你就是唐门的人了，进入唐门，你可以随时在我身边，那样的话，我也能随时给你做饭菜吃啊！而他只能吃一次哥哥做的饭菜，就让他多吃一点儿好了', 'said_by': '黎明'}, {'dialogue': '也是啊', 'said_by': '单翔'}, {'dialogue': '你认输吧', 'said_by': '夏子明'}, {'dialogue': '该认输的，是你才对', 'said_by': '黎明'}]}
{'summary': '- 夏子明要求黎明给他做饭，并

In [ ]:
save_datas = []
for data in all_datas:
    json_str = json.dumps(data["response"], ensure_ascii=False)
    conversations = [
        {"from":"human","value":data["input"]},
        {"from":"gpt","value":json_str}
    ]
    save_data = {
        "source":data["source"],
        "conversations":conversations,
        "system":task_prompt
    }
    save_datas.append(save_data)

In [ ]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero", if_skip = True):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    if os.path.exists( file_name ):
        if if_skip:
            print( "file ",file_name, "exists, skip" )
            return
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [ ]:
sample_and_save( save_datas, "dialogue_extract",if_skip = False )

负样本

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/CardBuild/dialogue_extract/negative

ls: cannot access '/content/drive/MyDrive/CardBuild/dialogue_extract/negative': No such file or directory


In [ ]:
import os
import json

# 指定文件夹路径
save_path = "/content/drive/MyDrive/CardBuild/dialogue_extract"

all_datas = []

count = 0
parsing_error_count = 0

# 遍历指定文件夹中的所有txt文件
for filename in os.listdir(save_path):
    if filename.endswith(".txt") and filename.startswith("negative"):
        # 构建完整的文件路径
        file_path = os.path.join(save_path, filename)

        error_count = 0

        # 打开并读取文件
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if error_count >3:
                    break
                try:
                    # 将每行解析为JSON
                    data = json.loads(line)

                    if "input" in data and "response" in data and "id" in data and "source" in data:
                        response_str = data['response']
                        try:
                            response = json.loads(response_str)
                            if "conversations" not in response:
                                continue

                            if len(response["conversations"]) > 0:
                                continue

                            # 将数据添加到列表中

                            all_datas.append({
                                "source": data["source"],
                                "id": data["id"],
                                "input": data["input"],
                                "response": response
                            })
                        except:
                            continue
                    else:
                        error_count += 1
                        continue
                    # break
                except json.JSONDecodeError:
                    # 如果解析JSON失败，跳过这一行
                    error_count += 1
                    continue


        # break

# 输出结果



In [ ]:
sample_and_save( all_datas, "dialogue_negative",sample_n = 20, if_skip = False )

In [ ]:
has_count = 0
ids = []
for id, data in enumerate( all_datas ):
    if len(data["response"]["conversations"])>0:
        has_count += 1
        ids.append(id)
print(has_count)

25
